# Compton scattering
Following example 7.4 in Griffths Introduction to particle physics

In [1]:
import sympy as sp
from sympy.physics.hep.gamma_matrices import GammaMatrix as G, LorentzIndex, gamma_trace, kahane_simplify
from sympy.tensor.tensor import tensor_indices, tensor_heads

import numpy as np

The amplitudes for the two lowest order feynmandiagrams for compton scattering are
\begin{align}
    \mathcal{M}_1 = g_e^2 \bar u^{s_4}(p_4) \gamma_\mu \epsilon_2^\mu \frac{\gamma_\nu p_1^\nu - \gamma_\nu p_3^\nu + m}{(p_1 - p_3)^2 - m^2}\gamma_\sigma {\epsilon_3^\sigma}^* u^{s_1}(p_1) \\
        \mathcal{M}_2 = g_e^2 \bar u^{s_4}(p_4) \gamma_\mu \epsilon_3^\mu \frac{\gamma_\nu p_1^\nu + \gamma_\nu p_2^\nu + m}{(p_1 + p_2)^2 - m^2}\gamma_\sigma {\epsilon_2^\sigma}^* u^{s_1}(p_1)
\end{align}

In [2]:
mu = np.array(tensor_indices("\mu_0:10", LorentzIndex))
p = np.array(tensor_heads("p_1:5", [LorentzIndex]))
m = sp.symbols("m")

p[0](mu[0]) * p[1](-mu[0]) * p[0](mu[1]) * p[1](-mu[1])

p_1(L_0)*p_2(-L_0)*p_1(L_1)*p_2(-L_1)

In [3]:
"""
A simple expression involving gamma matrices
"""

expr = np.prod([p[i](mu[i]) * G(-mu[i]) for i in range(4)])
expr

p_1(L_0)*GammaMatrix(-L_0)*p_2(L_1)*GammaMatrix(-L_1)*p_3(L_2)*GammaMatrix(-L_2)*p_4(L_3)*GammaMatrix(-L_3)

In [4]:
"""
Trace works without problems here
"""

gamma_trace(expr)

-4*p_1(L_0)*p_2(L_1)*p_3(-L_0)*p_4(-L_1) + 4*p_1(L_0)*p_2(-L_0)*p_3(L_1)*p_4(-L_1) + 4*p_1(L_0)*p_2(L_1)*p_3(-L_1)*p_4(-L_0)

In [5]:
"""
An expression containing both (contracted) gamma matrices an symbols
"""
expr = G(-mu[0]) * p[0](mu[0]) \
    * (G(-mu[1]) * p[1](mu[1]) + m)
expr

GammaMatrix(-L_0)*p_1(L_0)*(m + GammaMatrix(-L_1)*p_2(L_1))

This is where the SymPy-hep library starts to break. It is lacking crucial features, and is therefore unusable.

In [6]:
"""
The trace function fails internally, a clear bug
"""

gamma_trace(expr)

ValueError: all tensors must have the same indices

In [8]:
"""
Trying a hack by making sure there are equally many contracted indices in all parts of the sum
"""
g = LorentzIndex.metric
I = g(-mu[0], mu[0]) / 4

expr = G(-mu[0]) * p[0](mu[0]) * (G(mu[1])*p[1](-mu[1]) - m*I)
expr

GammaMatrix(-L_0)*p_1(L_0)*(GammaMatrix(L_1)*p_2(-L_1) + (-1/4)*m*metric(-L_1, L_1))

In [9]:
"""
Fails in the same way
"""

gamma_trace(expr)

ValueError: Repeated index: L_0

# Spin averaged value
Taking the average over all spins, we are left with
\begin{equation}
    |\mathcal{M}_1|^2 = \frac{g^4}{(p_1 - p_3)^2 - m^2} \mathrm{Tr} \left( \gamma_\mu (\gamma_\alpha p_1^\alpha - \gamma_\alpha p_3^\alpha + m) \gamma_\nu (\gamma_\beta p_1^\beta + m) \gamma^\nu (\gamma_\eta p_1^\eta - \gamma_\eta p_3^\eta + m) \gamma_\nu (\gamma_\rho p_4^\rho + m) \right)
\end{equation}

In [10]:
expr = G(-mu[0]) * ( G(-mu[1]) * p[0](mu[1]) - G(-mu[1])*p[2](mu[1]) + m)
expr *= G(-mu[2]) * (G(-mu[3]) * p[0](mu[3]) + m)
expr *= G(mu[2]) * ( G(-mu[4]) * p[0](mu[4]) - G(-mu[4])*p[2](mu[4]) + m) 
expr *= G(mu[0])
expr *= (G(-mu[5]) * p[3](mu[5]) + m)
expr

GammaMatrix(-L_4)*(m + GammaMatrix(-L_0)*p_1(L_0) + (-1)*GammaMatrix(-L_0)*p_3(L_0))*GammaMatrix(-L_2)*(m + GammaMatrix(-L_1)*p_1(L_1))*GammaMatrix(L_2)*(m + GammaMatrix(-L_3)*p_1(L_3) + (-1)*GammaMatrix(-L_3)*p_3(L_3))*GammaMatrix(L_4)*(m + GammaMatrix(-L_5)*p_4(L_5))

In [11]:
"""
This just breaks in a different ways. The library seems to be far from complete
"""

gamma_trace(expr)

TypeError: cannot add <class 'sympy.matrices.immutable.ImmutableDenseMatrix'> and <class 'sympy.core.numbers.Zero'>